In [1]:
import torch
print(torch.__version__)

1.7.0


In [3]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        
    def forward(self, x, h):
        new_h = torch.tanh(x + h)
        return new_h, new_h

my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h))
    

(tensor([[0.8582, 0.6030, 0.7003, 0.7795],
        [0.9014, 0.7901, 0.6871, 0.7846],
        [0.8939, 0.8152, 0.6666, 0.7386]]), tensor([[0.8582, 0.6030, 0.7003, 0.7795],
        [0.9014, 0.7901, 0.6871, 0.7846],
        [0.8939, 0.8152, 0.6666, 0.7386]]))


In [20]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)
        
    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h

my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(MyCell())
print(my_cell(x, h))

MyCell(
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
tensor([[ 0.3634,  0.3828,  0.5289,  0.3518],
        [ 0.3331,  0.6609,  0.4571,  0.2488],
        [ 0.4306,  0.8492, -0.1779,  0.4308]], grad_fn=<TanhBackward>)


In [18]:
class Test(torch.nn.Module):
    def __init__(self):
        super(Test, self).__init__()
        self.linear = torch.nn.Linear(4, 4)
    def __repr__(self):
        return 'in_features={}, out_features={}, bias={}'.format(
        "hello", "are you ok", "yes")
    

t = Test()
print(Test())

in_features=hello, out_features=are you ok, bias=yes


In [27]:
class MyDecisionGate(torch.nn.Module):
    def __init__(self, x):
        super(MyDecisionGate, self).__init__()
        self.x = x
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x
    def __repr__(self):
        return 'x = {}'.format(self.x)
        

class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.dg = MyDecisionGate(100)
        self.linear = torch.nn.Linear(4, 4)
        
    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h

my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(MyCell())
print(my_cell(x, h))

MyCell(
  (dg): x = 100
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
tensor([[0.7361, 0.3542, 0.5071, 0.2323],
        [0.8431, 0.4736, 0.3164, 0.7521],
        [0.7944, 0.4088, 0.5781, 0.0153]], grad_fn=<TanhBackward>)


In [32]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)
        
    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h

my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
print(traced_cell(x, h))
print(traced_cell.graph)
print(traced_cell.code)

MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
)
tensor([[ 0.0646,  0.1454,  0.3024,  0.8937],
        [ 0.3972, -0.0483,  0.6210,  0.6651],
        [ 0.3850,  0.4718,  0.0219,  0.2234]], grad_fn=<TanhBackward>)
graph(%self.1 : __torch__.___torch_mangle_15.MyCell,
      %input : Float(3:4, 4:1, requires_grad=0, device=cpu),
      %h : Float(3:4, 4:1, requires_grad=0, device=cpu)):
  %18 : __torch__.torch.nn.modules.linear.___torch_mangle_14.Linear = prim::GetAttr[name="linear"](%self.1)
  %20 : Tensor = prim::CallMethod[name="forward"](%18, %input)
  %12 : int = prim::Constant[value=1]() # <ipython-input-32-597991c5ae39>:7:0
  %13 : Float(3:4, 4:1, requires_grad=1, device=cpu) = aten::add(%20, %h, %12) # <ipython-input-32-597991c5ae39>:7:0
  %14 : Float(3:4, 4:1, requires_grad=1, device=cpu) = aten::tanh(%13) # <ipython-input-32-597991c5ae39>:7:0
  return (%14)

def forward(self,
    input: Tensor,
    h: Tensor) -> Tensor:
  _0 = torch.add((self.linear).forwar

In [50]:
class MyDecisionGate(torch.nn.Module):
    def __init__(self):
        super(MyDecisionGate, self).__init__()
        
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x
    def __repr__(self):
        return 'x = {}'.format("100")
        

class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)
        
    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h


x = torch.rand(3, 4)
h = torch.rand(3, 4)
#traced_cell = torch.jit.trace(my_cell, (x, h))
scripted_gate = torch.jit.script(MyDecisionGate())
my_cell = MyCell(scripted_gate)
traced_cell = torch.jit.script(my_cell)
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = (self.dg).forward((self.linear).forward(x, ), )
  new_h = torch.tanh(torch.add(_0, h, alpha=1))
  return (new_h, new_h)



In [51]:
class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCell(scripted_gate), (x, h))
    
    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h

rnn_loop = torch.jit.script(MyRNNLoop())
print(rnn_loop.code)

def forward(self,
    xs: Tensor) -> Tuple[Tensor, Tensor]:
  h = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  y = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  y0 = y
  h0 = h
  for i in range(torch.size(xs, 0)):
    _0 = (self.cell).forward(torch.select(xs, 0, i), h0, )
    y1, h1, = _0
    y0, h0 = y1, h1
  return (y0, h0)



In [52]:
class WrapRNN(torch.nn.Module):
    def __init__(self):
        super(WrapRNN, self).__init__()
        self.loop = torch.jit.script(MyRNNLoop())
    
    def forward(self, xs):
        y, h = self.loop(xs)
        return torch.relu(y)

traced = torch.jit.trace(WrapRNN(), (torch.rand(10, 3, 4)))
print(traced.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  _0, y, = (self.loop).forward(argument_1, )
  return torch.relu(y)



In [54]:
traced.save('wrapped_rnn.zip')
loaded = torch.jit.load('wrapped_rnn.zip')
print(loaded)
print(loaded.code)

RecursiveScriptModule(
  original_name=WrapRNN
  (loop): RecursiveScriptModule(
    original_name=MyRNNLoop
    (cell): RecursiveScriptModule(
      original_name=MyCell
      (dg): RecursiveScriptModule(original_name=MyDecisionGate)
      (linear): RecursiveScriptModule(original_name=Linear)
    )
  )
)
def forward(self,
    argument_1: Tensor) -> Tensor:
  _0, y, = (self.loop).forward(argument_1, )
  return torch.relu(y)

